## imports

In [1]:
{-# LANGUAGE PackageImports #-}
{-# LANGUAGE OverloadedStrings #-}
import "atidot-anonymizer" Atidot.Anonymizer.Monad
import qualified "atidot-anonymizer" Atidot.Anonymizer.Run as Anonymizer (runExample')
import "atidot-anonymizer" Atidot.Anonymizer.XML
import "atidot-anonymizer" Atidot.Anonymizer.Types
import "atidot-anonymizer" Atidot.Anonymizer.Utils
import "mtl"          Control.Monad.State (evalStateT)
import "data-default" Data.Default (def)

import Control.Monad
import qualified Data.Text as T
import qualified Data.ByteString.Lazy as BL
import qualified "bytestring" Data.ByteString.Lazy.Char8 as BL8


## Anonymization Paths

In [2]:
basePath = ["/","TXLife","TXLifeRequest" ,"OLifE"]

anonymizationPaths :: [Path]
anonymizationPaths = map (basePath ++)
        [["Holding", "Policy", "PolNumber"]
        ,["Holding", "Policy","OLifEExtension", "JournalEntry", "EntryAuthor"]
        ,["Holding", "Investment", "Banking"]
        ,[ "Party" ,"FullName"]
        ,[ "Party" ,"GovtID"]
        ,[ "Party" ,"Person", "FirstName"]
        ,[ "Party" ,"Person", "MiddleName"]
        ,[ "Party" ,"Person", "LastName"]
        ,[ "Party" ,"Person", "Prefix"]
        ,[ "Party" ,"Person", "Suffix"]
        ,[ "Party" ,"Address", "Line1"]
        ,[ "Party" ,"Address", "Line2"]
        ,[ "Party" ,"Address", "Line3"]
        ,[ "Party" ,"Address", "Line4"]
        ,[ "Party" ,"Phone" ]
        ,[ "Party" ,"EMailAddress" ]
        ,[ "FinancialStatement", "PolicyStatement", "PolNumber"]
        ]
                         
                   
xmlPath = "acord.xml"

## Example XML:
let's print the example xml used in this notebook

In [3]:
xmlData <- BL.readFile xmlPath
BL8.putStrLn $ BL.take 100 xmlData

<?xml version="1.0" encoding="UTF-8"?>
<TXLife xmlns="http://ACORD.org/Standards/Life/2" xmlns:xsi="

## Hashed Version

In [4]:
acordHashingExample :: Anonymizer ()
acordHashingExample = do
    whileNext $ do
        path <- current
        when (path `elem` anonymizationPaths) hash
    return ()    

execute `runExample` from Atidot.Anonymizer.Run, that prints two things:
1. XML after the script's traversal on the file
1. The paths traversed by the script

In [5]:
exp <- Anonymizer.runExample' xmlPath acordHashingExample

In [6]:
holdingPaths = map ((basePath ++ ["Holding"]) ++)
    [[ "Policy", "PolNumber"]
    ,[ "Policy", "OLifEExtension", "JournalEntry", "EntryAuthor"]
    ]


partyPaths = map ((basePath ++ ["Party"]) ++)
    [["FullName"]
    ,[ "GovtID"]
    ,[ "Person" , "FirstName"]
    ,[ "Person" , "MiddleName"]
    ,[ "Person" , "LastName"]
    ,[ "Person" , "Prefix"]
    ,[ "Person" , "Suffix"]
    ,[ "FinancialStatement", "PolicyStatement", "PolNumber"]
    ]

partyContactDetailPaths = map ((basePath ++ ["Party"]) ++)
    [[ "Address"]
    ,[ "Phone" ]
    ,[ "EMailAddress" ]
    ]

In [7]:
acordHashing :: Anonymizer ()
acordHashing = do
    let anonPaths = holdingPaths ++ partyPaths
    whileNext $ do
        path <- current
        when (path `elem` anonPaths) hash
        when (any (path =~) partyContactDetailPaths) hash
    return ()   

In [8]:
exp2 <- Anonymizer.runExample' xmlPath acordHashing


In [9]:
print $ exp2 == exp -- sanity check

True